#  Numerical solution of games against nature under uncertainty

## Theory

### Game Against Nature

A Game Against Nature is a 1-player game, in which a single rational self-interested player must choose a strategy, and the outcome and the player’s payoff depends on both his chosen strategy and the “choice” made by a totally disinterested nature.

### Payoff matrix 

Player has $m$ allowed types of actions (strategies), and nature can take $n$ different states. If the player chooses the strategy $i\space (i\in\overline{1,m})$, and nature turns out to be in the state $j\space (j\in\overline{1,n})$, then the player receives payoff $p_{ij}$. A payoff matrix is set up as a crosstabulation of player strategies and nature’s contingencies.

$$
P = \left(\begin{array}{cc} 
p_{11} & p_{12} & \dots & P_{1j} & \dots & p_{1n} \\
p_{21} & p_{22} & \dots & p_{2j} & \dots & p_{2n} \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
p_{i1} & p_{i2} & \dots & p_{ij} & \dots & p_{in} \\
p_{m1} & p_{m2} & \dots & p_{mj} & \dots & p_{mn} \\
\end{array}\right)
$$

### Cost matrix 

The cost matrix is similar to the payoff matrix but it contains costs instead of payoffs. Almost always the player wants to minimize costs or maximize payoffs.

$$
C = \left(\begin{array}{cc} 
c_{11} & c_{12} & \dots & c_{1j} & \dots & c_{1n} \\
c_{21} & c_{22} & \dots & c_{2j} & \dots & a_{2n} \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
c_{i1} & c_{i2} & \dots & c_{ij} & \dots & c_{in} \\
c_{m1} & c_{m2} & \dots & c_{mj} & \dots & c_{mn} \\
\end{array}\right)
$$

### Regret matrix

The regret matrix may be associated with a payoff or cost matrix. Let's assume nature take some state of $j$ (the $j$ column of payoffs or costs has been realized). In this situation, the player can choose one of his strategies, which is equivalent to choosing a certain row $i$.

In case of a payoff matrix there is a maximum among them $\alpha_j = \underset{i=1,\dots,m}{max} p_{ij}$. 
If the player selects a row other than the row of the maximum element, the payoff will be less, and the regret of the payoff can be calculated as the difference $r_{ij} = \alpha_j − p_{ij}$.

In case of a cost matrix there is a minimum among them $\beta_j = \underset{i=1,\dots,m}{min} c_{ij}$. 
If the player selects a row other than the row of the minimum element, the cost will be more, and the regret of the cost can be calculated as the difference $r_{ij} = c_{ij} - \beta_j$. 

The values of $r_{ij}$ are called regrets. A regret matrix is formed from them.

$$
R = \left(\begin{array}{cc} 
r_{11} & r_{12} & \dots & r_{1j} & \dots & r_{1n} \\
r_{21} & r_{22} & \dots & r_{2j} & \dots & r_{2n} \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
r_{i1} & r_{i2} & \dots & r_{ij} & \dots & r_{in} \\
r_{m1} & r_{m2} & \dots & r_{mj} & \dots & r_{mn} \\
\end{array}\right)
$$

# Code

## Base Code

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
from IPython.display import clear_output

# install missing imports
%pip install numpy
%pip install matplotlib
clear_output()

In [ ]:
import math
import re
import random
import numpy as np 
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from IPython.display import display, Latex

In [ ]:
class Game():
    def __init__(self):
        self._generate_random_game()
        self._generate_game_data()
    
    def _generate_costs(self):
        return np.random.randint(10, size  = self.shape)
        
    def _generate_payoffs(self):
        return np.random.randint(-10, 10, size = self.shape)

    def _generate_random_game(self):        
        self.shape = (random.randint(3, 5), random.randint(3, 5))        
        self.payoffs = self._generate_payoffs()
        self.costs = self._generate_costs()
        self.condition_chances = self._generate_probabilities()
    
    def _generate_game_data(self):        
        self.rows = self.shape[0]
        self.columns = self.shape[1]
        self.payoff_regret = self._calculate_payoff_regret()
        self.cost_regret = self._calculate_cost_regret()

    def _calculate_payoff_regret(self):
        maximum_payoffs_per_column = np.amax(self.payoffs, axis = 0)                        
        return np.reshape([maximum_payoffs_per_column - gain for gain in self.payoffs], self.shape)

    def _calculate_cost_regret(self):
        minimum_costs_per_column = np.amin(self.costs, axis = 0)                        
        return np.reshape([cost - minimum_costs_per_column for cost in self.costs], self.shape)

    def _generate_probabilities(self):
        list_of_random_floats = np.random.random(self.shape[1])
        sum_of_values = list_of_random_floats.sum()        
        normalized_values = list_of_random_floats / sum_of_values        
        return normalized_values

    def _is_probabilities_correct(self, probabilities : np.ndarray):        
        return math.isclose(probabilities.sum(), 1, abs_tol=10**-4) and len(probabilities) == self.columns
    
    def summary(self):
        print("Game Stats:")     
        self.print_probabilities()
        self.print_payoffs()      
        self.print_payoff_regret()
        self.print_costs()     
        self.print_cost_regret()                

    def print_payoffs(self):
        print(f"Payoffs: \n{self.payoffs}")
    
    def print_payoff_regret(self):
        print(f"Payoff Regret:\n{self.payoff_regret}")
    
    def print_costs(self):
        print(f"Costs: \n{self.costs}")

    def print_cost_regret(self):
        print(f"Cost Regret:\n{self.cost_regret}")
    
    def print_probabilities(self):
        print(f"Probabilities:\n{self.condition_chances}")


In [ ]:
class Criterion(ABC):
    def get_name(self) -> str:
        return re.sub(r"\B([A-Z])", r" \1", "%s" % type(self).__name__)               
    
    def _get_rows(self, m: np.ndarray):
        return m.shape[0]        

    def _get_columns(self, m: np.ndarray):
        return m.shape[1]
    
    def _is_dimensions_equal_two(m: np.ndarray):
        return len(m.shape) == 2

    def _get_equal_results(self, scores, best_score):
        return np.argwhere(scores == best_score).flatten()

    def _get_close_results(self, scores, best_score, abs_tol=10**-4):
        return np.argwhere(np.isclose(scores, best_score, atol=abs_tol)).flatten()    

    def _is_probabilities_correct(self, probabilities : np.ndarray):        
        return math.isclose(probabilities.sum(), 1, abs_tol=10**-4)        

In [ ]:
class PayoffCriterion(Criterion):
    @abstractmethod    
    def get_payoff_optimal_choices(self, payoffs: np.ndarray, probabilities: np.ndarray = None):
        raise NotImplementedError()

In [ ]:
class CostCriterion(Criterion):
    @abstractmethod    
    def get_cost_optimal_choices(self, costs: np.ndarray, probabilities: np.ndarray = None):
        raise NotImplementedError()

In [ ]:
class RegretCriterion(PayoffCriterion, CostCriterion):
    @abstractmethod    
    def get_regret_optimal_choices(self, regrets: np.ndarray, probabilities: np.ndarray = None):
        raise NotImplementedError()
    
    def calculate_payoff_regret(self, payoffs: np.ndarray):
        maxs = np.amax(payoffs, axis = 0) # per columns
        return np.reshape([maxs - payoff for payoff in payoffs], payoffs.shape)

    def calculate_cost_regret(self, costs: np.ndarray):
        mins = np.amin(costs, axis = 0) # per columns
        return np.reshape([cost - mins for cost in costs], costs.shape)

    def get_payoff_optimal_choices(self, payoffs: np.ndarray, probabilities: np.ndarray = None):
        regrets = self.calculate_payoff_regret(payoffs)
        return self.get_regret_optimal_choices(regrets, probabilities)

    def get_cost_optimal_choices(self, costs: np.ndarray, probabilities: np.ndarray = None):
        regrets = self.calculate_cost_regret(costs)
        return self.get_regret_optimal_choices(regrets, probabilities)    

In [ ]:
class FunctionsResearchCriterion(PayoffCriterion):
    def __init__(self):                      
        super().__init__() 
        self.coefficients = np.array([])  
        self._ylabel = "f(i, λ)"      

    @abstractmethod
    def function(self, *args):
        raise NotImplementedError()
    
    @abstractmethod
    def _get_coefficients(self, payoffs: np.ndarray, probabilities: np.ndarray = None):
        raise NotImplementedError() 
    
    def get_payoff_optimal_choices(self, payoffs: np.ndarray, probabilities: np.ndarray = None):                                        
        self.coefficients = self._get_coefficients(payoffs, probabilities)
        count = np.zeros(shape = self._get_rows(payoffs), dtype=np.int32)                     
        # could've just compare coefficients or solve analytically
        points = np.linspace(0, 1, 1000)
        for p in points:            
            fn_results = [self.function(p, *c) for c in self.coefficients]
            max_fn_result = np.amax(fn_results)
            idx = self._get_equal_results(fn_results, max_fn_result)
            
            for index in idx:
                count[index] += 1
                  
        max_count = np.amax(count)
        return self._get_equal_results(count, max_count)
        
    @abstractmethod
    def _get_tex_fn(self, *args) -> str:
        raise NotImplementedError()

    def print_functions(self):                
        for index, c in enumerate(self.coefficients):
            if IN_COLAB:
                print(f"Strategy №{index + 1} function:")
                display(Latex(f"${self._get_tex_fn(*c)}$"))   
            else:
                display(Latex(f"Strategy №{index + 1} function: ${self._get_tex_fn(*c)}$"))   
    
    def plot(self):
        fig = plt.figure()
        ax = plt.axes()        
        plt.title(self.get_name())
        plt.xlabel("λ")
        plt.ylabel(self._ylabel)              
        l = np.linspace(0, 1, 2)
        for index, c in enumerate(self.coefficients):        
            plt.plot(l, self.function(l, *c), color=np.random.random(3), label= "Strategy №%d" % (index + 1)) 
        plt.legend()
        plt.draw()
          
    def summary(self):        
        self.print_functions()
        self.plot()    

In [ ]:
class CriteriaTester():
    def test_payoff(self, criterion: Criterion, game: Game):        
        if isinstance(criterion, PayoffCriterion):
            result =criterion.get_payoff_optimal_choices(game.payoffs, game.condition_chances)
            self.print_result(criterion, result)

    def test_cost(self, criterion: Criterion, game: Game):        
        if isinstance(criterion, CostCriterion):
            result = criterion.get_cost_optimal_choices(game.costs, game.condition_chances)
            self.print_result(criterion, result)

    def print_result(self, criterion: Criterion, choices) -> str:        
        if len(choices) > 1:
            print(f"According to {criterion.get_name()}, the optimal strategies are №{choices + 1}.")
        else:
            print(f"According to {criterion.get_name()}, the optimal strategy is №{choices[0] + 1}.")

In [ ]:
class GameCriteriaPicksStats():
    def __init__(self, game: Game) -> None:
        self.criteria = []
        self.game = game
        pass

    def add_criterion(self, criterion: Criterion):
        self.criteria.append(criterion)            

    def get_payoff_choices(self):        
        choices = []
        for criterion in self.criteria:
            if isinstance(criterion, PayoffCriterion):
                choices += (criterion.get_payoff_optimal_choices(self.game.payoffs, self.game.condition_chances) + 1).tolist()
        return choices
            
    def get_cost_choices(self):        
        choices = []
        for criterion in self.criteria:
            if isinstance(criterion, CostCriterion):
                choices += (criterion.get_cost_optimal_choices(self.game.costs, self.game.condition_chances) + 1).tolist()
        return choices

    def payoff_hist(self):
        choices = self.get_payoff_choices()
        self.hist(choices, "Strategy picks by payoff criteria")

    def cost_hist(self):
        choices = self.get_cost_choices()
        self.hist(choices, "Strategy picks by cost criteria")

    def hist(self, choices, title = "Strategy picks"):
        # Set up the bins centered on the integers, i.e. -0.5, 0.5, 1,5, 2.5, ... up to max(data) + 1.5. Then substract -0.5 to eliminate the extra bin at the end.
        bins = np.arange(1, self.game.rows + 1.5) - 0.5

        fig, ax = plt.subplots()
        counts, edges, bars = ax.hist(choices, bins, density=False, label="Picks") # density=True would make frequencies
        ax.set_xticks(bins + 0.5)        
    
        #plt.bar_label(bars)
        plt.ylabel('Number of Picks')
        plt.xlabel('Strategy Number');        
        plt.legend(loc="upper left")
        plt.title(title)
        
    def summary(self):        
        self.payoff_hist()
        self.cost_hist()

## Criteria

### Criterion of Rationality

#### Laplace Criterion (Bayes' Criterion)

$$\underset{i}{max} \space \frac{1}{n} \sum_{j=1}^n p_{ij} \space\space\space\space (i \in \overline{1,m}, \space j \in \overline{1,n})$$
$$\underset{i}{min} \space \frac{1}{n} \sum_{j=1}^n c_{ij} \space\space\space\space (i \in \overline{1,m}, \space j \in \overline{1,n})$$

In [ ]:
class LaplaceCriterion(PayoffCriterion, CostCriterion):
    def get_payoff_optimal_choices(self, payoffs: np.ndarray, probabilities: np.ndarray = None):        
        sums = self._calculate_sums(payoffs)
        max_expected_payoff = np.amax(sums)        
        return self._get_equal_results(sums, max_expected_payoff)
    
    def get_cost_optimal_choices(self, costs: np.ndarray, probabilities: np.ndarray = None):        
        sums = self._calculate_sums(costs)
        min_expected_cost = np.amin(sums)        
        return self._get_equal_results(sums, min_expected_cost)
        
    def _calculate_sums(self, m: np.ndarray):
        return np.array([np.sum(row) for row in m]) / self._get_columns(m)       

#### Maximum Expected Payoff Criterion (Expected Utility Maximization)

$$\underset{i}{max} \sum_{j=1}^n p_{ij} y_{j} \space (i \in \overline{1,m})$$

In [ ]:
class MaximumExpectedPayoffCriterion(PayoffCriterion):
    def get_payoff_optimal_choices(self, payoffs: np.ndarray, probabilities: np.ndarray):          
        sums = [np.sum(payoff * probabilities) for payoff in payoffs]
        maxsum = np.amax(sums)        
        return self._get_equal_results(sums, maxsum)

#### Minimum Expected Cost Criterion (Expected Cost Minimization)

$$\underset{i}{min} \sum_{j=1}^n c_{ij} y_{j} \space (i \in \overline{1,m})$$

In [ ]:
class MaximumExpectedCostCriterion(CostCriterion):
    def get_cost_optimal_choices(self, costs: np.ndarray, probabilities: np.ndarray = None):        
        sums = [np.sum(cost * probabilities) for cost in costs]
        minsum = np.amin(sums)        
        return self._get_equal_results(sums, minsum)

#### Minimum Expected Regret Criterion (Expected Regret Minimization)

$$\underset{i}{min} \sum_{j=1}^n r_{ij} y_{j} \space (i \in \overline{1,m})$$

In [ ]:
class MinimumExpectedRegretCriterion(RegretCriterion):
    def get_regret_optimal_choices(self, regrets: np.ndarray, probabilities: np.ndarray):
        sums = [np.sum(regret * probabilities) for regret in regrets]
        minsum = np.amin(sums)      
        return self._get_equal_results(sums, minsum)        

### Criterion of Pessimism

#### Wald Criterion

$$\underset{i}{max} \cdot \underset{j}{min} \cdot p_{ij} \space\space\space\space (i \in \overline{1,m}, \space j \in \overline{1,n})$$
$$\underset{i}{min} \cdot \underset{j}{max} \cdot c_{ij} \space\space\space\space (i \in \overline{1,m}, \space j \in \overline{1,n})$$

In [ ]:
class WaldCriterion(PayoffCriterion, CostCriterion):
    def get_payoff_optimal_choices(self, payoffs: np.ndarray, probabilities: np.ndarray = None):        
        mins = np.amin(payoffs, axis = 1)        
        maximin = np.amax(mins)        
        return self._get_equal_results(mins, maximin)

    def get_cost_optimal_choices(self, costs: np.ndarray, probabilities: np.ndarray = None):        
        maxs = np.amax(costs, axis = 1)        
        minimax = np.amin(maxs)        
        return self._get_equal_results(maxs, minimax)             

#### Savage Criterion (Minimax Regret Criterion)

$$\underset{i}{min} \cdot \underset{j}{max} \cdot r_{ij} \space\space\space\space (i \in \overline{1,m}, \space j \in \overline{1,n})$$

In [ ]:
class SavageCriterion(RegretCriterion):
    def get_regret_optimal_choices(self, regrets: np.ndarray, probabilities: np.ndarray = None):
        maxs = np.amax(regrets, axis = 1) # per rows
        minimax = np.amin(maxs)        
        return self._get_equal_results(maxs, minimax)

### Criterion of Optimism

#### Maximax Criterion

$$\underset{i}{max} \cdot \underset{j}{max} \cdot p_{ij} \space\space\space\space (i \in \overline{1,m}, \space j \in \overline{1,n})$$

In [ ]:
class MaximaxCriterion(PayoffCriterion):
    def get_payoff_optimal_choices(self, payoffs: np.ndarray, probabilities: np.ndarray = None):        
        maxs = np.amax(payoffs, axis = 1)
        maximax = np.amax(maxs)        
        return self._get_equal_results(maxs, maximax)

#### Minimin Criterion

$$\underset{i}{min} \cdot \underset{j}{min} \cdot c_{ij} \space\space\space\space (i \in \overline{1,m}, \space j \in \overline{1,n})$$

In [ ]:
class MiniminCriterion(CostCriterion):
    def get_cost_optimal_choices(self, costs: np.ndarray, probabilities: np.ndarray = None):        
        mins = np.amin(costs, axis = 1)
        minimin = np.amin(mins)   
        return self._get_equal_results(mins, minimin) 

### Criterion of Realism

#### Hurwicz Criterion (Criterion of Realism)

$$g(i, \lambda) = \lambda \space \underset{j}{min} \space p_{ij} + (1-\lambda) \space \underset{j}{max} \space p_{ij} \space \space (i \in \overline{1,m})$$

In [ ]:
class HurwiczCriterion(FunctionsResearchCriterion):
    def function(self, l, min, max):
        return l * min + (1 - l) * max
        
    def _get_coefficients(self, payoffs: np.ndarray, probabilities: np.ndarray = None):        
        return [(np.min(payoff), np.max(payoff)) for payoff in payoffs]
        
    def _get_tex_fn(self, min, max) -> str:        
        return f"g(i,\lambda)={min}\lambda+{max}(1-\lambda)" if max >= 0 else f"g(i,\lambda)={min}\lambda{max}(1-\lambda)"                

#### Hodges–Lehmann Criterion

$$h(i, \lambda) = \lambda \space \underset{j}{min} \space p_{ij} + (1-\lambda) \space \sum_{j=1}^n \space p_{ij} \space y_{j} \space \space (i \in \overline{1,m})$$

In [ ]:
class HodgesLehmannCriterion(FunctionsResearchCriterion):
    def function(self, l, min, sum):
        return l * min + (1 - l) * sum

    def get_payoff_optimal_choices(self, payoffs: np.ndarray, probabilities: np.ndarray):
        return super().get_payoff_optimal_choices(payoffs, probabilities)

    def _get_coefficients(self, payoffs: np.ndarray, probabilities: np.ndarray):        
        return [(np.min(payoff), np.sum(payoff * probabilities)) for payoff in payoffs]
                            
    def _get_tex_fn(self, min, sum) -> str:        
        return f"h(i,\lambda)={min}\lambda+{sum}(1-\lambda)" if sum >= 0 else f"h(i,\lambda)={min}\lambda{sum}(1-\lambda)"

#### Hodges-Lehmann Criterion (Modification 1)

$$u(i, \lambda) = \lambda \space \underset{j}{max} \space p_{ij} + (1-\lambda) \space \sum_{j=1}^n \space p_{ij} \space y_{j} \space \space (i \in \overline{1,m})$$

In [ ]:
class HodgesLehmannCriterion_Modification1(FunctionsResearchCriterion):
    def function(self, l, max, sum):
        return l * max + (1 - l) * sum

    def _get_coefficients(self, payoffs: np.ndarray, probabilities: np.ndarray):
        return [(np.max(payoff), np.sum(payoff * probabilities)) for payoff in payoffs]
                            
    def _get_tex_fn(self, max, sum) -> str:        
        return f"h(i,\lambda)={max}\lambda+{sum}(1-\lambda)" if sum >= 0 else f"h(i,\lambda)={max}\lambda{sum}(1-\lambda)"

#### Hodges-Lehmann Criterion (Modification 2)

$$v(i, \space \lambda, \space \mu) = \lambda \space \underset{j}{min} \space p_{ij} + \mu \space \underset{j}{max} \space p_{ij} + (1-\lambda-\mu) \space \sum_{j=1}^n \space p_{ij} \space y_{j}\space \space (i \in \overline{1,m})$$

In [ ]:
class HodgeLemanCriterion_Modification2(FunctionsResearchCriterion):
    def _get_coefficients(self, payoffs: np.ndarray, probabilities: np.ndarray):
        return [(np.min(payoff), np.max(payoff), np.sum(payoff * probabilities)) for payoff in payoffs]
                
    def function(self, l, u, min, max, sum):
        return (l * min) + (u * max) + (1 - l - u) * sum

    def get_payoff_optimal_choices(self, payoffs: np.ndarray, probabilities: np.ndarray):        
        self.coefficients = self._get_coefficients(payoffs, probabilities)
        count = np.zeros(shape = self._get_rows(payoffs), dtype=np.int32)
        _l = np.linspace(0, 1, 100)
        _u = np.linspace(0, 1, 100)
        l = []
        u = []                  
        for lp in _l:            
            for up in _u:
                if lp + up < 1:
                    l.append(lp)
                    u.append(up)
                    fns_results = [self.function(lp, up, *c) for c in self.coefficients]                    
                    max_fn_result = np.amax(fns_results)
                    idx = self._get_equal_results(fns_results, max_fn_result)
                    for index in idx:
                        count[index] += 1
        
        self.l = np.array(l)
        self.u = np.array(u)                                                         
        max_count = np.amax(count)
        return self._get_equal_results(count, max_count)

    def _get_tex_fn(self, min, max, sum) -> str:
        if max >= 0 and sum >= 0:
            return f"v(i,\space\lambda,\space\mu)={min}\lambda+{max}\mu+{sum}(1-\lambda-\mu)"
        elif max >= 0:
            return f"v(i,\space\lambda,\space\mu)={min}\lambda+{max}\mu{sum}(1-\lambda-\mu)"
        else:
            return f"v(i,\space\lambda,\space\mu)={min}\lambda{max}\mu{sum}(1-\lambda-\mu)"

    def plot(self):        
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        plt.title(self.get_name())
        ax.set_xlabel('l')
        ax.set_ylabel('u')
        ax.set_zlabel('u(i, λ, μ)')
                                                  
        for index, c in enumerate(self.coefficients):                                                      
            ax.plot(self.l, self.u, self.function(self.l, self.u, *c), label= "Strategy №%d" % (index + 1))            
        
        plt.legend()
        ax.view_init(10, 30)
    

## Tests

In [ ]:
laplace = LaplaceCriterion()
maximax = MaximaxCriterion()
minimin = MiniminCriterion()
wald = WaldCriterion()
savage = SavageCriterion()
max_utility = MaximumExpectedPayoffCriterion()
min_cost = MaximumExpectedCostCriterion()
min_risk = MinimumExpectedRegretCriterion()
hurwicz = HurwiczCriterion()
hodge_leman = HodgesLehmannCriterion()
hodge_leman_m1 = HodgesLehmannCriterion_Modification1()
hodge_leman_m2 = HodgeLemanCriterion_Modification2()

criteria = [laplace, max_utility, min_cost, min_risk, wald, savage, maximax, minimin, hurwicz, hodge_leman, hodge_leman_m1, hodge_leman_m2]

game = Game()
game.summary()

In [ ]:
tester = CriteriaTester()
for criterion in criteria:
    tester.test_payoff(criterion, game)

In [ ]:
for criterion in criteria:
    tester.test_cost(criterion, game)

In [ ]:
hurwicz.summary()

In [ ]:
hodge_leman.summary()

In [ ]:
hodge_leman_m1.summary()

In [ ]:
hodge_leman_m2.summary()

## Statistics

In [ ]:
stats = GameCriteriaPicksStats(game)

for criterion in criteria:
    stats.add_criterion(criterion)

In [ ]:
stats.summary()